# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LORA
* Model: GPT2
* Evaluation approach: AutoModelForSequenceClassification 
* Fine-tuning dataset: "yelp_review_full"

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = load_dataset("yelp_review_full", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=19
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'text'],
    num_rows: 520000
})

In [2]:
[len(x) for x in dataset['train']['text']]

[426,
 1244,
 341,
 312,
 453,
 359,
 401,
 679,
 331,
 407,
 1532,
 3850,
 1413,
 986,
 1825,
 448,
 861,
 433,
 2544,
 2287,
 466,
 89,
 75,
 900,
 1707,
 255,
 768,
 294,
 1010,
 540,
 1071,
 587,
 1179,
 1031,
 384,
 1053,
 632,
 724,
 247,
 242,
 438,
 617,
 872,
 2349,
 508,
 1005,
 1086,
 1140,
 159,
 3053,
 621,
 624,
 103,
 1350,
 469,
 664,
 1662,
 761,
 986,
 1556,
 476,
 374,
 132,
 1023,
 520,
 1799,
 758,
 455,
 2144,
 3369,
 477,
 1186,
 707,
 1646,
 624,
 826,
 994,
 1906,
 762,
 638,
 541,
 1562,
 121,
 553,
 316,
 1124,
 308,
 1216,
 652,
 853,
 512,
 1185,
 115,
 351,
 271,
 1257,
 792,
 594,
 1431,
 586,
 4196,
 734,
 360,
 1376,
 164,
 304,
 696,
 876,
 641,
 1101,
 670,
 580,
 921,
 1211,
 210,
 178,
 1806,
 364,
 396,
 978,
 1366,
 236,
 452,
 676,
 732,
 454,
 822,
 364,
 783,
 674,
 1136,
 1128,
 439,
 1231,
 1692,
 82,
 97,
 3027,
 543,
 769,
 161,
 2599,
 348,
 126,
 847,
 368,
 748,
 1078,
 825,
 1458,
 868,
 1960,
 326,
 1788,
 266,
 92,
 1917,
 1075,
 1056

In [3]:
from transformers import AutoTokenizer
from transformers import GPT2TokenizerFast


tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.add_special_tokens({"pad_token": "[PAD]"})

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, max_length=100, truncation=True, return_tensors='pt')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/520000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 520000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 130000
    })
})

In [6]:
small_train= tokenized_datasets['train'].shuffle(seed=100).select(range(10000))
small_test= tokenized_datasets['test'].shuffle(seed=100).select(range(1000))

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=5)

for param in model.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [15]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",per_device_train_batch_size = 1,
        per_device_eval_batch_size = 1, num_train_epochs=1,save_strategy='epoch', logging_steps=10)

In [18]:
import torch

print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 499183 KiB | 499183 KiB | 499183 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |      0 B   |
|       from small pool |  12783 KiB |  12783 KiB |  12783 KiB |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         | 499183 KiB | 499183 KiB | 499183 KiB |      0 B   |
|       from large pool | 486400 KiB | 486400 KiB | 486400 KiB |

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [19]:
trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 9.522686004638672,
 'eval_accuracy': 0.208,
 'eval_runtime': 17.8327,
 'eval_samples_per_second': 56.077,
 'eval_steps_per_second': 56.077}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [20]:
from peft import LoraConfig
config = LoraConfig(task_type="SEQ_CLS")

In [21]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [22]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [23]:
print(lora_model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()

In [24]:
lora_model.print_trainable_parameters()

trainable params: 302,592 || all params: 124,742,400 || trainable%: 0.24257349545944282


In [25]:
small_train = small_train.rename_column('label','labels')
small_test = small_test.rename_column('label','labels')

In [26]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",per_device_train_batch_size = 1,
        per_device_eval_batch_size = 1, num_train_epochs=1,save_strategy='epoch', save_steps=5)

In [27]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.298100,1.243794,0.457000


Checkpoint destination directory test_trainer/checkpoint-10000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=10000, training_loss=1.6284845581054688, metrics={'train_runtime': 538.3117, 'train_samples_per_second': 18.577, 'train_steps_per_second': 18.577, 'total_flos': 512151552000000.0, 'train_loss': 1.6284845581054688, 'epoch': 1.0})

In [29]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()

In [31]:
import os
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /workspace


In [38]:
lora_model.save_pretrained("gpt-lora-final")
#my_lora_model = AutoPeftModelForCausalLM.from_pretrained("gpt-lora-part2")

In [39]:
from peft import PeftModelForSequenceClassification

saved_model_directory="/workspace/gpt-lora-final"

#PeftModelForSequenceClassification.from_pretrained(saved_model_directory)

In [41]:
PeftModelForSequenceClassification.from_pretrained(lora_model,saved_model_directory)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): PeftModelForSequenceClassification(
      (base_model): LoraModel(
        (model): GPT2ForSequenceClassification(
          (transformer): GPT2Model(
            (wte): Embedding(50257, 768)
            (wpe): Embedding(1024, 768)
            (drop): Dropout(p=0.1, inplace=False)
            (h): ModuleList(
              (0-11): 12 x GPT2Block(
                (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
                (attn): GPT2Attention(
                  (c_attn): Linear(
                    in_features=768, out_features=2304, bias=True
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [43]:
peft_model=PeftModelForSequenceClassification.from_pretrained(lora_model,'gpt-lora-final')

In [44]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [45]:
trainer.evaluate()

{'eval_loss': 9.522686004638672,
 'eval_accuracy': 0.208,
 'eval_runtime': 21.1109,
 'eval_samples_per_second': 47.369,
 'eval_steps_per_second': 47.369}